In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive/')
'/content/drive/My Drive/Study/modelCheckpoint/____________________k46_boston_{epoch:02d}-{val_loss:.4f}.hdf5'
import pandas as pd
import numpy as np
import os
import glob
import random
import warnings
import tensorflow.keras.backend as K
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, load_model,Model
from tensorflow.keras.layers import Dense, Conv1D, Dropout, MaxPool1D,Flatten, Reshape, LSTM,Input,concatenate,BatchNormalization,LayerNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from lightgbm import LGBMRegressor
from tensorflow.keras.optimizers import Adam


# 파일 불러오기
train = pd.read_csv('/content/drive/My Drive/Study/데이콘/태양열/data/train/train.csv')
submission = pd.read_csv('/content/drive/My Drive/Study/데이콘/태양열/data/sample_submission.csv')

#1. DATA

# GHI column 추가 함수
def Add_features(data):
    c = 243.12
    b = 17.62
    gamma = (b * (data['T']) / (c + (data['T']))) + np.log(data['RH'] / 100)
    dp = ( c * gamma) / (b - gamma)
    data.insert(1,'Td',dp)
    data.insert(1,'T-Td',data['T']-data['Td'])
    data.insert(1,'GHI',data['DNI']+data['DHI'])

    sunup=[]
    for i in range(len(data)):
        temp = data.iloc[i,-1]
        if temp>0.0:
            sunup.append(1)
        else :
            sunup.append(0)
    data['Sun_up']=sunup

    # Time 추가
    Time=[]
    for i in range(len(data)):
        temp = 60*data.iloc[i,1]+data.iloc[i,2]
        Time.append(temp)
    data['Time']=Time


    return data

# train data column정의
def preprocess_data(data, is_train=True):
    c = 243.12
    b = 17.62
    gamma = (b * (data['T']) / (c + (data['T']))) + np.log(data['RH'] / 100)
    dp = ( c * gamma) / (b - gamma)
    data.insert(1,'Td',dp)
    data.insert(1,'T-Td',data['T']-data['Td'])
    data.insert(1,'GHI',data['DNI']+data['DHI'])
    sunup=[]
    for i in range(len(data)):
        temp = data.iloc[i,-1]
        if temp>0.0:
            sunup.append(1)
        else :
            sunup.append(0)
    data['Sun_up']=sunup
    # Time 추가
    Time=[]
    for i in range(len(data)):
        temp = 60*data.iloc[i,1]+data.iloc[i,2]
        Time.append(temp)
    data['Time']=Time
    temp = data.copy()
    temp = temp[['Hour','TARGET','T-Td','DHI','DNI','WS','RH','T']]

    if is_train==True:          
        temp['Target1'] = temp['TARGET'].shift(-48).fillna(method='ffill')   # 다음날의 Target
        temp['Target2'] = temp['TARGET'].shift(-48*2).fillna(method='ffill') # 다다음날의 Target
        temp = temp.dropna()    # 결측값 제거
        return temp.iloc[:-96]  # 뒤에서 이틀은 뺀다. (예측하고자 하는 날짜이기 때문)

    elif is_train == False:
        temp = temp[['Hour','TARGET','T-Td','DHI','DNI','WS','RH','T']]

        return temp.iloc[-48:, :]

df_train = preprocess_data(train)
# print(df_train.shape)   # (52464, 10)
# print(df_train[:48])

#-------------test 함수정의
df_test = []
for i in range(81):
    file_path = '/content/drive/My Drive/Study/데이콘/태양열/data/test/' + str(i) + '.csv'
    temp = pd.read_csv(file_path)
    temp = preprocess_data(temp, is_train=False)
    df_test.append(temp)

df_test = pd.concat(df_test)
test = np.array(df_test)
print(df_test.shape) # (3888, 8)
x_test = df_test.to_numpy()

###=====================================================================
# 시계열 데이터(함수) --> y1, y2 로 나눠 각각 훈련시키기
def split_xy(dataset, time_steps) :
    x, y1, y2 = [],[],[]
    for i in range(len(dataset)) :
        x_end = i + time_steps
        if x_end > len(dataset) :
            break
        tmp_x = dataset[i:x_end, :-2] # ['Hour' ~ 'T']
        tmp_y1 = dataset[x_end-1:x_end,-2] # Target1
        tmp_y2 = dataset[x_end-1:x_end,-1]   # Target2
        x.append(tmp_x)
        y1.append(tmp_y1)
        y2.append(tmp_y2)
    return np.array(x), np.array(y1), np.array(y2)

X = df_train.to_numpy()
# print(X.shape)      # (52464, 10)
x,y1,y2 = split_xy(X,1)
print(x.shape, y1.shape, y2.shape) #(52464, 1, 8) (52464, 1) (52464, 1)

#####===========전처리
x_train, x_val, y1_train, y1_val, y2_train, y2_val = train_test_split(x,y1,y2,
                           train_size = 0.7,shuffle = False, random_state = 0)

# print(x_train.shape,x_val.shape) #(36724, 1, 8) (15740, 1, 8)
print(y1_train.shape, y1_val.shape, y2_train.shape, y2_val.shape) #(36724, 1) (15740, 1) (36724, 1) (15740, 1)
x_train= x_train.reshape(36724*1, 8)
x_val= x_val.reshape(15740*1, 8)

scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
x_val = scaler.transform(x_val)

x_train= x_train.reshape(36724, 1, 8)
x_val= x_val.reshape(15740, 1, 8)
x_test= x_test.reshape(3888, 1, 8)
# print(x_train.shape,x_val.shape,x_test.shape) #(36724, 1, 8) (15740, 1, 8) (3888, 1, 8)

# 함수 : Quantile loss definition
def quantile_loss(q, y_true, y_pred):
    err = (y_true - y_pred)
    return K.mean(K.maximum(q*err, (q-1)*err), axis=-1)

q = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

#2. 모델링

def modeling() :
    inputs = Input(shape=(x_train.shape[1], x_train.shape[2]))
    adam = Adam(learning_rate=0.003)

    lstm8 = LSTM(256,activation='relu',return_sequences=True)(inputs)
    lstm8 = LSTM(256,activation='relu',return_sequences=True)(lstm8)
    lstm8 = LSTM(128,activation='relu',return_sequences=True)(lstm8)
    lstm8 = LSTM(128,activation='relu',return_sequences=True)(lstm8)
    lstm8 = LSTM(64,activation='relu',return_sequences=True)(lstm8)
    lstm8 = LSTM(64,activation='relu',return_sequences=True)(lstm8)
    lstm8 = LSTM(32,activation='relu',return_sequences=True)(lstm8)
    lstm8 = LSTM(16,activation='relu',return_sequences=True)(lstm8)
    lstm8 = LSTM(8,activation='relu')(lstm8)
    lstm8 = LayerNormalization()(lstm8)

    # lstm7 = LSTM(256,activation='relu',return_sequences=True)(inputs)
    # lstm7 = LSTM(256,activation='relu',return_sequences=True)(lstm7)
    # lstm7 = LSTM(128,activation='relu',return_sequences=True)(lstm7)
    # lstm7 = LSTM(128,activation='relu',return_sequences=True)(lstm7)
    # lstm7 = LSTM(64,activation='relu',return_sequences=True)(lstm7)
    # lstm7 = LSTM(32,activation='relu',return_sequences=True)(lstm7)
    # lstm7 = LSTM(16,activation='relu',return_sequences=True)(lstm7)
    # lstm7 = LSTM(8,activation='relu')(lstm7)
    # lstm7 = LayerNormalization()(lstm7)

    # lstm6 = LSTM(256,activation='relu',return_sequences=True)(inputs)
    # lstm6 = LSTM(128,activation='relu',return_sequences=True)(lstm6)
    # lstm6 = LSTM(128,activation='relu',return_sequences=True)(lstm6)
    # lstm6 = LSTM(64,activation='relu',return_sequences=True)(lstm6)
    # lstm6 = LSTM(32,activation='relu',return_sequences=True)(lstm6)
    # lstm6 = LSTM(16,activation='relu',return_sequences=True)(lstm6)
    # lstm6 = LSTM(8,activation='relu')(lstm6)
    # lstm6 = LayerNormalization()(lstm6)

    # lstm3 = LSTM(256,activation='relu',return_sequences=True)(inputs)
    # lstm3 = LSTM(128,activation='relu',return_sequences=True)(lstm3)
    # lstm3 = LSTM(64,activation='relu',return_sequences=True)(lstm3)
    # lstm3 = LSTM(16,activation='relu')(lstm3)
    # lstm3 = LayerNormalization()(lstm3)

    # lstm4 = LSTM(256,activation='relu',return_sequences=True)(inputs)
    # lstm4 = LSTM(128,activation='relu',return_sequences=True)(lstm4)
    # lstm4 = LSTM(64,activation='relu',return_sequences=True)(lstm4)
    # lstm4 = LSTM(32,activation='relu',return_sequences=True)(lstm4)
    # lstm4 = LSTM(16,activation='relu')(lstm4)
    # lstm4 = LayerNormalization()(lstm4)

    # lstm5 = LSTM(256,activation='relu',return_sequences=True)(inputs)
    # lstm5 = LSTM(128,activation='relu',return_sequences=True)(lstm5)
    # lstm5 = LSTM(64,activation='relu',return_sequences=True)(lstm5)
    # lstm5 = LSTM(32,activation='relu',return_sequences=True)(lstm5)
    # lstm5 = LSTM(16,activation='relu',return_sequences=True)(lstm5)
    # lstm5 = LSTM(8,activation='relu')(lstm5)
    # lstm5 = LayerNormalization()(lstm5)
    
    # lstm = LSTM(256,activation='relu',return_sequences=True)(inputs)
    # lstm = LSTM(64,activation='relu',return_sequences=True)(lstm)
    # lstm = LSTM(16,activation='relu')(lstm)
    # lstm = LayerNormalization()(lstm)
    
    # lstm1 = LSTM(256,activation='relu',return_sequences=True)(inputs)
    # lstm1 = LSTM(16,activation='relu')(lstm1)
    # lstm1 = LayerNormalization()(lstm1)
    
    # lstm2 = LSTM(16,activation='relu')(inputs)
    # lstm2 = LayerNormalization()(lstm2)
    
    
    #concat = concatenate([lstm3,lstm4,lstm5,lstm,lstm1,lstm2,lstm6,lstm7,lstm8])
    dense = Dense(128,activation='relu')(lstm8)
    dense = Dense(64,activation='relu')(dense)
    dense = Dense(32,activation='relu')(dense)
    dense = Dense(8,activation='relu')(dense)
    outputs = Dense(1,activation='relu')(dense)
    model = Model(inputs=inputs,outputs=outputs)
    return model

# 3. 컴파일, 훈련
#####========컴파일,  훈련, predict
#y1, y2 각각 저장하기

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
es = EarlyStopping(monitor = 'val_loss', patience = 20)
lr = ReduceLROnPlateau(monitor = 'val_loss', patience = 7, factor = 0.75, verbose = 1)
epochs = 20
bs = 1024

x = []
for i in [0.7,0.8,0.9]:
    if q not in [0.5]:
        print("\n\n ",i,"\n\n")
        model = modeling()
        filepath_cp = f'/content/drive/My Drive/Study/데이콘/태양열/colab4/dacon_y1_quantile_{i:.1f}.hdf5'
        cp = ModelCheckpoint(filepath_cp,save_best_only=True,monitor = 'val_loss')
        model.compile(loss = lambda y_true,y_pred: quantile_loss(i,y_true,y_pred), optimizer = 'adam', metrics = [lambda y,y_pred: quantile_loss(i,y,y_pred)])
        model.fit(x_train,y1_train,epochs = epochs, batch_size = bs, validation_data = (x_val,y1_val),callbacks = [es,cp,lr])
        pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
        x.append(pred)
    else:
        print("\n\n ",i,"\n\n")
        model = modeling()
        filepath_cp = f'/content/drive/My Drive/Study/데이콘/태양열/colab4/dacon_y1_quantile_{i:.1f}.hdf5'
        cp = ModelCheckpoint(filepath_cp,save_best_only=True,monitor = 'val_loss')
        model.compile(loss = 'mae', optimizer = 'adam')
        model.fit(x_train,y1_train,epochs = epochs, batch_size = bs, validation_data = (x_val,y1_val),callbacks = [es,cp,lr])
        pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
        x.append(pred)
df_temp1 = pd.concat(x, axis = 1)
df_temp1[df_temp1<0] = 0
num_temp1 = df_temp1.to_numpy()
submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = num_temp1

x = []
# 모레!!
for i in q:
    if q not in [0.5]:
        print("\n\n ",i,"\n\n")
        model = modeling()
        filepath_cp = f'/content/drive/My Drive/Study/데이콘/태양열/colab4/dacon_y2_quantile_{i:.1f}.hdf5'
        cp = ModelCheckpoint(filepath_cp,save_best_only=True,monitor = 'val_loss')
        model.compile(loss = lambda y_true,y_pred: quantile_loss(i,y_true,y_pred), optimizer = 'adam', metrics = [lambda y,y_pred: quantile_loss(i,y,y_pred)])
        model.fit(x_train,y2_train,epochs = epochs, batch_size = bs, validation_data = (x_val,y2_val),callbacks = [es,cp,lr])
        pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
        x.append(pred)
    else:
        print("\n\n ",i,"\n\n")
        model = modeling()
        filepath_cp = f'/content/drive/My Drive/Study/데이콘/태양열/colab4/dacon_y2_quantile_{i:.1f}.hdf5'
        cp = ModelCheckpoint(filepath_cp,save_best_only=True,monitor = 'val_loss')
        model.compile(loss = 'mae', optimizer = 'adam')
        model.fit(x_train,y2_train,epochs = epochs, batch_size = bs, validation_data = (x_val,y2_val),callbacks = [es,cp,lr])
        pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
        x.append(pred)
df_temp2 = pd.concat(x, axis = 1)
df_temp2[df_temp2<0] = 0
num_temp2 = df_temp2.to_numpy()
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = num_temp2

##??????????????근데 이 지표가 잘됐다는거 평가하는 방법있을까?????????  로스밖에 없어
submission.to_csv('/content/drive/My Drive/Study/데이콘/태양열/colab4/upgrade_sub_0125_colab2.csv', index = False)
##제출후 등수 135  -- 1.94653

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
(3888, 8)
(52464, 1, 8) (52464, 1) (52464, 1)
(36724, 1) (15740, 1) (36724, 1) (15740, 1)


  0.7 


Epoch 1/20
36/36 [==============================] - 9s 60ms/step - loss: 12.1382 - <lambda>: 12.1382 - val_loss: 12.7498 - val_<lambda>: 12.7498
Epoch 2/20
36/36 [==============================] - 1s 32ms/step - loss: 11.5515 - <lambda>: 11.5515 - val_loss: 9.5426 - val_<lambda>: 9.5426
Epoch 3/20
36/36 [==============================] - 1s 31ms/step - loss: 7.9991 - <lambda>: 7.9991 - val_loss: 5.4986 - val_<lambda>: 5.4986
Epoch 4/20
36/36 [==============================] - 1s 32ms/step - loss: 4.1498 - <lambda>: 4.1498 - val_loss: 3.1250 - val_<lambda>: 3.1250
Epoch 5/20
36/36 [==============================] - 1s 32ms/step - loss: 3.0371 - <lambda>: 3.0371 - val_loss: 2.7292 - val_<lambda>: 2.7292
Epoch 6/20
36/36 [==============================] - 1s 32

ValueError: ignored

In [38]:
import os
from google.colab import drive
drive.mount('/content/drive/')
'/content/drive/My Drive/Study/modelCheckpoint/____________________k46_boston_{epoch:02d}-{val_loss:.4f}.hdf5'
import pandas as pd
import numpy as np
import os
import glob
import random
import warnings
import tensorflow.keras.backend as K
warnings.filterwarnings("ignore")

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential, load_model,Model
from tensorflow.keras.layers import Dense, Conv1D, Dropout, MaxPool1D,Flatten, Reshape, LSTM,Input,concatenate,BatchNormalization,LayerNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from lightgbm import LGBMRegressor
from tensorflow.keras.optimizers import Adam


# 파일 불러오기
train = pd.read_csv('/content/drive/My Drive/Study/데이콘/태양열/data/train/train.csv')
submission = pd.read_csv('/content/drive/My Drive/Study/데이콘/태양열/data/sample_submission.csv')

# train data column정의
def preprocess_data(data, is_train=True):
    c = 243.12
    b = 17.62
    gamma = (b * (data['T']) / (c + (data['T']))) + np.log(data['RH'] / 100)
    dp = ( c * gamma) / (b - gamma)
    data.insert(1,'Td',dp)
    data.insert(1,'T-Td',data['T']-data['Td'])
    data.insert(1,'GHI',data['DNI']+data['DHI'])
    sunup=[]
    for i in range(len(data)):
        temp = data.iloc[i,-1]
        if temp>0.0:
            sunup.append(1)
        else :
            sunup.append(0)
    data['Sun_up']=sunup
    # Time 추가
    Time=[]
    for i in range(len(data)):
        temp = 60*data.iloc[i,1]+data.iloc[i,2]
        Time.append(temp)
    data['Time']=Time
    temp = data.copy()
    temp = temp[['Hour','TARGET','T-Td','DHI','DNI','WS','RH','T']]

    if is_train==True:          
        temp['Target1'] = temp['TARGET'].shift(-48).fillna(method='ffill')   # 다음날의 Target
        temp['Target2'] = temp['TARGET'].shift(-48*2).fillna(method='ffill') # 다다음날의 Target
        temp = temp.dropna()    # 결측값 제거
        return temp.iloc[:-96]  # 뒤에서 이틀은 뺀다. (예측하고자 하는 날짜이기 때문)

    elif is_train == False:
        temp = temp[['Hour','TARGET','T-Td','DHI','DNI','WS','RH','T']]

        return temp.iloc[-48:, :]

df_train = preprocess_data(train)
df_test = []
for i in range(81):
    file_path = '/content/drive/My Drive/Study/데이콘/태양열/data/test/' + str(i) + '.csv'
    temp = pd.read_csv(file_path)
    temp = preprocess_data(temp, is_train=False)
    df_test.append(temp)

df_test = pd.concat(df_test)
test = np.array(df_test)
print(df_test.shape) # (3888, 8)
x_test = df_test.to_numpy()
def split_xy(dataset, time_steps) :
    x, y1, y2 = [],[],[]
    for i in range(len(dataset)) :
        x_end = i + time_steps
        if x_end > len(dataset) :
            break
        tmp_x = dataset[i:x_end, :-2] # ['Hour' ~ 'T']
        tmp_y1 = dataset[x_end-1:x_end,-2] # Target1
        tmp_y2 = dataset[x_end-1:x_end,-1]   # Target2
        x.append(tmp_x)
        y1.append(tmp_y1)
        y2.append(tmp_y2)
    return np.array(x), np.array(y1), np.array(y2)

X = df_train.to_numpy()
# print(X.shape)      # (52464, 10)
x,y1,y2 = split_xy(X,1)
print(x.shape, y1.shape, y2.shape) #(52464, 1, 8) (52464, 1) (52464, 1)

#####===========전처리
x_train, x_val, y1_train, y1_val, y2_train, y2_val = train_test_split(x,y1,y2,
                           train_size = 0.7,shuffle = False, random_state = 0)

# print(x_train.shape,x_val.shape) #(36724, 1, 8) (15740, 1, 8)
print(y1_train.shape, y1_val.shape, y2_train.shape, y2_val.shape) #(36724, 1) (15740, 1) (36724, 1) (15740, 1)
x_train= x_train.reshape(36724*1, 8)
x_val= x_val.reshape(15740*1, 8)

scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
x_val = scaler.transform(x_val)

x_train= x_train.reshape(36724, 1, 8)
x_val= x_val.reshape(15740, 1, 8)
x_test= x_test.reshape(3888, 1, 8)
# print(x_train.shape,x_val.shape,x_test.shape) #(36724, 1, 8) (15740, 1, 8) (3888, 1, 8)

# 함수 : Quantile loss definition
def quantile_loss(q, y_true, y_pred):
    err = (y_true - y_pred)
    return K.mean(K.maximum(q*err, (q-1)*err), axis=-1)

q = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
# 3. 컴파일, 훈련
#####========컴파일,  훈련, predict
#y1, y2 각각 저장하기

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
es = EarlyStopping(monitor = 'val_loss', patience = 10)
lr = ReduceLROnPlateau(monitor = 'val_loss', patience = 5, factor = 0.9, verbose = 1)
epochs = 200
bs = 256

# 내일!!
x = []
for i in q:
    if q not in [0.4,0.5,0.6]:
        print("\n\n ",i,"\n\n")
        filepath_cp = f'/content/drive/My Drive/Study/데이콘/태양열/colab4/dacon_y1_quantile_{i:.1f}.hdf5'
        model = load_model(filepath_cp,compile=False)
        pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
        x.append(pred)
    else:
        print("\n\n ",i,"\n\n")
        filepath_cp = f'/content/drive/My Drive/Study/데이콘/태양열/colab4/dacon_y1_quantile_{i:.1f}.hdf5'
        model = load_model(filepath_cp,compile=False)
        pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
        x.append(pred)
df_temp1 = pd.concat(x, axis = 1)
df_temp1[df_temp1<0] = 0
num_temp1 = df_temp1.to_numpy()
submission.loc[submission.id.str.contains("Day7"), "q_0.1":] = num_temp1

x = []
# 모레!!
for i in q:
    if q not in [0.4,0.5,0.6]:
        print("\n\n ",i,"\n\n")
        filepath_cp = f'/content/drive/My Drive/Study/데이콘/태양열/colab4/dacon_y2_quantile_{i:.1f}.hdf5'
        model = load_model(filepath_cp,compile=False)
        pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
        x.append(pred)
    else:
        print("\n\n ",i,"\n\n")
        filepath_cp = f'/content/drive/My Drive/Study/데이콘/태양열/colab4/dacon_y2_quantile_{i:.1f}.hdf5'
        model = load_model(filepath_cp,compile=False)
        pred = pd.DataFrame(model.predict(x_test).reshape(3888,1).round(2))
        x.append(pred)
df_temp2 = pd.concat(x, axis = 1)
df_temp2[df_temp2<0] = 0
num_temp2 = df_temp2.to_numpy()
submission.loc[submission.id.str.contains("Day8"), "q_0.1":] = num_temp2

##??????????????근데 이 지표가 잘됐다는거 평가하는 방법있을까?????????  로스밖에 없어
submission.to_csv('/content/drive/My Drive/Study/데이콘/태양열/colab4/upgrade_sub_0125_colab.csv', index = False)
##제출후 등수 135  -- 1.94653












def split_x(data,x_row,x_col):
    x = []
    for i in range(len(data)-x_row+1):
        subset = data[i:(i+x_row),0:x_col]
        x.append(subset)
    return np.array(x)

def split_y(data,y_row,y_col):
    y = []
    for i in range(len(data)-y_row+1):
        subset = data[(i):(i+y_row),y_col:-1]
        y.append(subset)
    return np.array(y)

x_col,x_row,y_col,y_row = 9,336,-3,96


# ['Day', 'Hour', 'Minute', 'DHI', 'DNI', 'WS', 'RH', 'T', 'TARGET','Sun_up]
drop_columns = ['Day','Hour','Minute']
df = pd.read_csv('/content/drive/My Drive/Study/데이콘/태양열/data/train/train.csv')

# sunup 추가
sunup=[]
for i in range(len(df)):
    temp = df.iloc[i,-1]
    if temp>0.0:
        sunup.append(1)
    else :
        sunup.append(0)
df['Sun_up']=sunup

# Time 추가
Time=[]
for i in range(len(df)):
    temp = 60*df.iloc[i,1]+df.iloc[i,2]
    Time.append(temp)
df['Time']=Time

y = split_y(df.values,y_row,y_col)
y1=y[:,:,0].reshape(-1,96,)
y2=y[:,:,1].reshape(-1,96,)


df.drop(drop_columns,axis='columns',inplace=True)
datasets = df.values[:,:]
scaler = StandardScaler()
datasets = scaler.fit_transform(datasets)

x = split_x(datasets,x_row,x_col)[:-96]



modelpath_2 = '/content/drive/My Drive/Study/데이콘/태양열/model_binary.h5'




binary = []
result=[]
for i in range(81):
    # 데이터 로드
    filename = '/content/drive/My Drive/Study/데이콘/태양열/data/test/{}.csv'.format(i)
    df=pd.read_csv(filename)
    # sun_up 추가
    ghi=[]
    sun_up=[]
    Time=[]
    for i in range(len(df)):
        temp = np.cos(np.pi/2-np.abs(df.iloc[i,1]%12 - 6)/6*np.pi/2)
        ghi.append(temp)
        temp = 60*df.iloc[i,1]+df.iloc[i,2]
        Time.append(temp)
        temp = df.iloc[i,-1]
        if temp>0.0:
            sun_up.append(1)
        else :
            sun_up.append(0)
    df['Sun_up']=sun_up
    df['Time']=Time
    df.drop(drop_columns,axis='columns',inplace=True)
    datasets = df.values[:,:]
    datasets = scaler.transform(datasets)
    binary.append(datasets[:,-2].reshape(7,48))
binary = np.array(binary)
model2 = load_model(modelpath_2)
model2.summary()
print(binary.shape)
binary_result = np.round(model2.predict(binary)).reshape(-1,)
print(binary_result[0])
print(binary_result[1])
print(binary_result[2])
print(binary_result[3])
print(binary_result.shape)

submit = pd.read_csv('/content/drive/My Drive/Study/데이콘/태양열/colab4/upgrade_sub_0125_colab.csv',index_col=0)
print(submit)
for i in [1,2,3,4,5,6,7,8,9]:
    submit.iloc[:,i-1]=submit.iloc[:,i-1]*binary_result
submit.to_csv('/content/drive/My Drive/Study/데이콘/태양열/colab4/upgrade_sub_0125_colab.csv')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
(3888, 8)
(52464, 1, 8) (52464, 1) (52464, 1)
(36724, 1) (15740, 1) (36724, 1) (15740, 1)


  0.1 




  0.2 




  0.3 




  0.4 




  0.5 




  0.6 




  0.7 




  0.8 




  0.9 




  0.1 




  0.2 




  0.3 




  0.4 




  0.5 




  0.6 




  0.7 




  0.8 




  0.9 


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 64)                21888     
_________________________________________________________________
dense (Dense)                (None, 512)               33280     
_________________________________________________________________
dense_1 (Dense)              (None, 96)                49248     
Total params: 104,416
Trainable params: 104,416
Non-trainable params: 0
___________________________